# Washington Working Families Tax Credit

In 2008, the Washington State Legislature enacted the Working Families Tax Credit (WFTC, also called the Working Families Tax Exemption), but it was not funded until 2021, when Governor Jay Inslee signed [HB 1297](https://app.leg.wa.gov/billsummary?BillNumber=1297&Year=2021&Initiative=false) / [SB 5387](https://app.leg.wa.gov/billsummary?BillNumber=5387&Chamber=Senate&Year=2021).
The WFTC provides up to \$1,200 to filers who are eligible for the Earned Income Tax Credit, or who would be if their children had Social Security Numbers rather than Individual Taxpayer Identification Numbers.
Washington will deliver the first WFTC payments in 2023, based on 2022 tax returns.

## Examples

In [2]:
from policyengine_us import IndividualSim
import pandas as pd
import plotly.express as px

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}


def make_wftc(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members)
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="WA")
    sim.vary("employment_income", max=60_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            wa_wftc=sim.calc("wa_working_families_tax_credit")[0].round(),
            mtr=-sim.deriv(
                "wa_working_families_tax_credit",
                "employment_income",
                wrt_target="head",
            ),
            adults=adults,
            children=str(children),
        )
    )


# Make a table of WFTCs for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make_wftc(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    mtr="Marginal tax rate of Washington WFTC",
    adults="Adults",
    children="Children",
    wa_wftc="Washington WFTC",
)

fig = px.line(
    df,
    "employment_income",
    "wa_wftc",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Washington Working Families Tax Credit",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

The WA WFTC creates marginal tax rates ranging from 12 to 18 percent, plus downward spikes at the first dollar when households become eligible, and spikes reflecting the eligibility cliff, either from the minimum benefit to zero, or from some other amount to no longer being eligible for the federal EITC.


In [5]:
fig = px.line(
    df,
    "employment_income",
    "mtr",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Washington Working Families Tax Credit marginal tax rate",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat=".0%",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
    yaxis_range=[0, 0.3],
)
fig.show()

## Budgetary impact

Applying the Washington WFTC logic to the 2020 Current Population Survey March Supplement shows that the program would have cost an estimated \$213 million in 2020 (paid out in 2021).

[The Washington Department of Revenue estimated that the bill would provide \$236 million in refund payouts in FY2023](https://fnspublic.ofm.wa.gov/FNSPublicSearch/GetPDF?packageID=63327#page=6) (i.e., reflecting income from 2022).


In [3]:
from policyengine_us import Microsimulation

ms = Microsimulation(dataset_year=2020)
(ms.calc("wa_working_families_tax_credit", period=2022).sum() / 1e6).round()

213.0